In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

In [ ]:
cluster = Cluster(ClusterConfiguration(
    name='gptfttest',
    namespace='default',
    num_workers=2,
    min_cpus=2,
    max_cpus=2,
    min_memory=8,
    max_memory=8,
    num_gpus=1,
    instascale=True, #<---instascale enabled
    machine_types=["m5.xlarge", "g4dn.xlarge"],
))

In [ ]:
cluster.up()

In [ ]:
cluster.wait_ready()

In [ ]:
cluster.details()

In [ ]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--do_train",
    "--do_eval",
    "--output_dir", "/opt/app-root/src,
    "--overwrite_output_dir"
]

In [ ]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="gpt_og.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements_gpt.txt"}
)
job = jobdef.submit(cluster)

In [ ]:
job.status()

Retrieve raw log output at anytime with:

In [ ]:
job.logs()

View live updates for status, logs, and other information with:

In [ ]:
cluster.cluster_dashboard_uri()

In [ ]:
job.status()

Once the fine-tuning process is complete, the model file should show up on the left sidebar (if not, click on the folder icon that will take you to /opt/app-root/stc directory of the container). You would need to follow three steps:
1. Download the model file into your local laptop.
2. Convert the model to caikit format by following instructions outlined [here](https://github.com/opendatahub-io/caikit-tgis-serving/blob/main/demo/kserve/built-tip.md).
3. Containerize your model into a MinIO bucket by following instructions outlined [here](https://github.com/opendatahub-io/caikit-tgis-serving/blob/main/demo/kserve/create-minio.md). 

Once your converted model is in a MinIO container available at a quay.io registry, continue executing the cells below for serving of the model.

In [ ]:
cluster.down()

In [ ]:
!oc create namespace demo-serving
!oc patch smmr/default -n istio-system --type='json' -p="[{'op': 'add', 'path': '/spec/members/-', 'value': \"demo-serving\"}]"

In [ ]:
# Instructions on converting the model

In [ ]:
!oc apply -f caikit-servingruntime-demo.yaml -n demo-serving

In [ ]:
!oc apply -f caikit-isvc-demo.yaml -n demo-serving

In [ ]:
!oc get isvc/caikit-example-isvc -n demo-serving

'/opt/app-root/src/codeflare-sdk/demo-notebooks/guided-demos'

In [ ]:
!export KSVC_HOSTNAME=$(oc get ksvc caikit-example-isvc-predictor -n demo-serving -o jsonpath='{.status.url}' | cut -d'/' -f3)
!grpcurl -insecure -d '{"text": "This demo is awesome because"}' -H "mm-model-id: gpt2-caikit" ${KSVC_HOSTNAME}:443 caikit.runtime.Nlp.NlpService/TextGenerationTaskPredict

In [ ]:
auth.logout()